# Try joining L and R features into 1 feature

In [ ]:
df.head()

REF ALT   AF  HIAF  HICNT  VD     SN  ADJAF   L_A   L_T  ...  \
CHROM POS                                                             ...   
chr1  826893   G   A 1.00  1.00     40  40  80.00   0.05 35.00  5.00  ...   
      941119   A   G 1.00  1.00      8   8  16.00   0.38 15.00  5.00  ...   
      944168   G   T 0.05  0.05      2   2   4.00   0.00 10.00 30.00  ...   
      944858   A   G 1.00  1.00     67  67 134.00   0.01 35.00 15.00  ...   
      946247   G   A 0.95  1.00     19  19  38.00   0.00 25.00 25.00  ...   

              R_TTAA  R_GTAA  R_TATC  R_HOMO_POLY_A  R_HOMO_POLY_T  \
CHROM POS                                                            
chr1  826893       0       0       0              0              0   
      941119       0       0       0              0              0   
      944168       0       0       0              3              0   
      944858       0       0       0              0              0   
      946247       0       0       0              0              0   

              R_HOMO_POLY_G  R_HOMO_POLY_C  R_PALINDROME  R_HAIRPIN  \
CHROM POS                                                             
chr1  826893              3              3             0          0   
      941119              0              5             0          0   
      944168              4              0             4          0   
      944858              0              0             4          0   
      946247              3              4             6          0   

              IS_ARTIFACT  
CHROM POS                  
chr1  826893        False  
      941119        False  
      944168         True  
      944858        False  
      946247        False  

[5 rows x 541 columns]

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1834281 entries, ('chr1', '826893') to ('chrX', '143712338')
Columns: 541 entries, REF to IS_ARTIFACT
dtypes: bool(1), float64(12), int64(526), object(2)
memory usage: 7.4+ GB


In [ ]:
# Add content of each base
for nt in ['A', 'T', 'C', 'G']:
    df[nt] = df['L_'+nt] + df['R_'+nt]
#df['A'] = df['L_A'] + df['R_A']

In [ ]:
df.info() # +4

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1834281 entries, ('chr1', '826893') to ('chrX', '143712338')
Columns: 545 entries, REF to G
dtypes: bool(1), float64(16), int64(526), object(2)
memory usage: 7.5+ GB


In [ ]:
# Max homopoly size of each base
for nt in ['A', 'T', 'C', 'G']:
    df['HOMO_POLY_'+nt] = df[['L_HOMO_POLY_'+nt, 'R_HOMO_POLY_'+nt]].max(axis=1)

In [ ]:
df.info() # +4

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1834281 entries, ('chr1', '826893') to ('chrX', '143712338')
Columns: 549 entries, REF to HOMO_POLY_G
dtypes: bool(1), float64(16), int64(530), object(2)
memory usage: 7.5+ GB


In [ ]:
# Max palindrome size 
df['PALINDROME'] = df[['L_PALINDROME', 'R_PALINDROME']].max(axis=1)

In [10]:
df.info() # +1

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1834281 entries, ('chr1', '826893') to ('chrX', '143712338')
Columns: 550 entries, REF to PALINDROME
dtypes: bool(1), float64(16), int64(531), object(2)
memory usage: 7.5+ GB


In [11]:
# Max hairpin loop size 
df['HAIRPIN'] = df[['L_HAIRPIN', 'R_HAIRPIN']].max(axis=1)

In [12]:
df.info() # +1

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1834281 entries, ('chr1', '826893') to ('chrX', '143712338')
Columns: 551 entries, REF to HAIRPIN
dtypes: bool(1), float64(16), int64(532), object(2)
memory usage: 7.5+ GB


In [13]:
# Add each kmer type
nt = ['A', 'T', 'C', 'G']
kmers = [n1+n2+n3+n4 for n1 in nt for n2 in nt for n3 in nt for n4 in nt] 
# looks like: ['AAAA', 'AAAT', 'AAAC',..]

for kmer in kmers:
    df[kmer] = df['L_'+kmer] + df['R_'+kmer]

In [14]:
df.info() # +256

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1834281 entries, ('chr1', '826893') to ('chrX', '143712338')
Columns: 807 entries, REF to GGGG
dtypes: bool(1), float64(16), int64(788), object(2)
memory usage: 11.0+ GB


In [15]:
df.head()

REF ALT   AF  HIAF  HICNT  VD     SN  ADJAF   L_A   L_T  ...  \
CHROM POS                                                             ...   
chr1  826893   G   A 1.00  1.00     40  40  80.00   0.05 35.00  5.00  ...   
      941119   A   G 1.00  1.00      8   8  16.00   0.38 15.00  5.00  ...   
      944168   G   T 0.05  0.05      2   2   4.00   0.00 10.00 30.00  ...   
      944858   A   G 1.00  1.00     67  67 134.00   0.01 35.00 15.00  ...   
      946247   G   A 0.95  1.00     19  19  38.00   0.00 25.00 25.00  ...   

              GGTC  GGTG  GGCA  GGCT  GGCC  GGCG  GGGA  GGGT  GGGC  GGGG  
CHROM POS                                                                 
chr1  826893     0     0     0     0     0     0     1     0     0     0  
      941119     0     0     0     0     0     1     0     0     1     3  
      944168     0     1     0     0     1     0     0     2     1     3  
      944858     0     0     0     0     0     0     0     0     0     0  
      946247     1     1     0     1     0     0     0     1     0     0  

[5 rows x 807 columns]

In [18]:
# Remove all the L and R features since we have combined features now

# Remove L and R content of each base
for nt in ['A', 'T', 'C', 'G']:
    df.drop(columns=['L_'+nt, 'R_'+nt], axis=1, inplace=True)

In [19]:
df.info() # -8

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1834281 entries, ('chr1', '826893') to ('chrX', '143712338')
Columns: 799 entries, REF to GGGG
dtypes: bool(1), float64(8), int64(788), object(2)
memory usage: 10.9+ GB


In [20]:
# Remove L and R homopoly size of each base
for nt in ['A', 'T', 'C', 'G']:
    df.drop(columns=['L_HOMO_POLY_'+nt, 'R_HOMO_POLY_'+nt], axis=1, inplace=True)

In [21]:
df.info() # -8

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1834281 entries, ('chr1', '826893') to ('chrX', '143712338')
Columns: 791 entries, REF to GGGG
dtypes: bool(1), float64(8), int64(780), object(2)
memory usage: 10.8+ GB


In [22]:
# Remove L and R palindrome size 
df.drop(columns=['L_PALINDROME', 'R_PALINDROME'], axis=1, inplace=True)

In [23]:
df.info() # -2

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1834281 entries, ('chr1', '826893') to ('chrX', '143712338')
Columns: 789 entries, REF to GGGG
dtypes: bool(1), float64(8), int64(778), object(2)
memory usage: 10.8+ GB


In [24]:
# Remove L and R hairpin loop size 
df.drop(columns=['L_HAIRPIN', 'R_HAIRPIN'], axis=1, inplace=True)

In [25]:
df.info() # -2

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1834281 entries, ('chr1', '826893') to ('chrX', '143712338')
Columns: 787 entries, REF to GGGG
dtypes: bool(1), float64(8), int64(776), object(2)
memory usage: 10.8+ GB


In [26]:
# Remove L and R of each kmer type
for kmer in kmers:
    df.drop(columns=['L_'+kmer, 'R_'+kmer], axis=1, inplace=True)

In [27]:
df.info() # -512

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1834281 entries, ('chr1', '826893') to ('chrX', '143712338')
Columns: 275 entries, REF to GGGG
dtypes: bool(1), float64(8), int64(264), object(2)
memory usage: 3.8+ GB


In [29]:
df.to_pickle('ML_df_joined_LR_feats.pkl') 

# Try joining L and R features into 1 feature BY ADDING

In [21]:
df = pd.read_pickle('ML_df.pkl')

In [22]:
df.head()

REF ALT   AF  HIAF  HICNT  VD     SN  ADJAF   L_A   L_T  ...  \
CHROM POS                                                             ...   
chr1  826893   G   A 1.00  1.00     40  40  80.00   0.05 35.00  5.00  ...   
      941119   A   G 1.00  1.00      8   8  16.00   0.38 15.00  5.00  ...   
      944168   G   T 0.05  0.05      2   2   4.00   0.00 10.00 30.00  ...   
      944858   A   G 1.00  1.00     67  67 134.00   0.01 35.00 15.00  ...   
      946247   G   A 0.95  1.00     19  19  38.00   0.00 25.00 25.00  ...   

              R_TTAA  R_GTAA  R_TATC  R_HOMO_POLY_A  R_HOMO_POLY_T  \
CHROM POS                                                            
chr1  826893       0       0       0              0              0   
      941119       0       0       0              0              0   
      944168       0       0       0              3              0   
      944858       0       0       0              0              0   
      946247       0       0       0              0              0   

              R_HOMO_POLY_G  R_HOMO_POLY_C  R_PALINDROME  R_HAIRPIN  \
CHROM POS                                                             
chr1  826893              3              3             0          0   
      941119              0              5             0          0   
      944168              4              0             4          0   
      944858              0              0             4          0   
      946247              3              4             6          0   

              IS_ARTIFACT  
CHROM POS                  
chr1  826893        False  
      941119        False  
      944168         True  
      944858        False  
      946247        False  

[5 rows x 541 columns]

In [23]:
# Add content of each base
for nt in ['A', 'T', 'C', 'G']:
    df[nt] = df['L_'+nt] + df['R_'+nt]

In [24]:
# Add homopoly size of each base
for nt in ['A', 'T', 'C', 'G']:
    df['HOMO_POLY_'+nt] = df['L_HOMO_POLY_'+nt] + df['R_HOMO_POLY_'+nt]

In [30]:
# Max palindrome size 
df['PALINDROME'] = df['L_PALINDROME'] + df['R_PALINDROME']

In [26]:
# Add hairpin loop size 
df['HAIRPIN'] = df['L_HAIRPIN'] + df['R_HAIRPIN']

In [27]:
# Add each kmer type
nt = ['A', 'T', 'C', 'G']
kmers = [n1+n2+n3+n4 for n1 in nt for n2 in nt for n3 in nt for n4 in nt] 
# looks like: ['AAAA', 'AAAT', 'AAAC',..]

for kmer in kmers:
    df[kmer] = df['L_'+kmer] + df['R_'+kmer]

In [32]:
# Remove all the L and R features since we have combined features now

# Remove L and R content of each base
for nt in ['A', 'T', 'C', 'G']:
    df.drop(columns=['L_'+nt, 'R_'+nt], axis=1, inplace=True)

In [33]:
# Remove L and R homopoly size of each base
for nt in ['A', 'T', 'C', 'G']:
    df.drop(columns=['L_HOMO_POLY_'+nt, 'R_HOMO_POLY_'+nt], axis=1, inplace=True)

In [34]:
# Remove L and R palindrome size 
df.drop(columns=['L_PALINDROME', 'R_PALINDROME'], axis=1, inplace=True)

In [35]:
# Remove L and R hairpin loop size 
df.drop(columns=['L_HAIRPIN', 'R_HAIRPIN'], axis=1, inplace=True)

In [ ]:
# Remove L and R of each kmer type
for kmer in kmers:
    df.drop(columns=['L_'+kmer, 'R_'+kmer], axis=1, inplace=True)

In [ ]:
df.info() # 275

In [ ]:
df.to_pickle('ML_df_add_LR_feats.pkl') 